In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

Loading random walk csv file and creating dataframe to store the outputs

In [ ]:
data = pd.read_csv("random_walk.csv", delimiter=',')
random_walk_predictions = pd.DataFrame()

Applying Random Forest model (random walk data)

In [ ]:
# Split the data into features (X) and target variable (y)
X = data[['period']]
y = data['demand']

# Fit the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# Generate forecast for the next 48 observations
forecast_dates = pd.DataFrame({'period': range(397, 445)})
forecasted_demand = rf_model.predict(forecast_dates)
random_walk_predictions['Random_Forest'] = forecasted_demand

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(data['period'], data['demand'], label='Original Data')
plt.plot(forecast_dates, forecasted_demand, color='green', label='Forecast')
plt.xlabel('Period')
plt.ylabel('Demand')
plt.legend()
plt.show()


Aplying XGBoost model (random walk data)

In [ ]:
# Split the data into features (X) and target variable (y)
X = data[['period']]
y = data['demand']

# Fit the model
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X, y)

# Generate forecast for the next 48 observations
forecast_dates = pd.DataFrame({'period': range(397, 445)})
forecasted_demand = xgb_model.predict(forecast_dates)
random_walk_predictions['XGBoost'] = forecasted_demand

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(data['period'], data['demand'], label='Original Data')
plt.plot(forecast_dates, forecasted_demand, color='green', label='Forecast')
plt.xlabel('Period')
plt.ylabel('Demand')
plt.legend()
plt.show()

Applying SARIMA model (random walk data)

In [ ]:
# Fit auto_arima to automatically select the best ARIMA model
model = auto_arima(data['demand'], seasonal=True, m=12)

# Fit the best ARIMA model
order = model.get_params()['order']
seasonal_order = model.get_params()['seasonal_order']
arima_model = ARIMA(data['demand'], order=order, seasonal_order=seasonal_order)
arima_model_fit = arima_model.fit()

# Generate forecast for the next 48 observations
forecast_dates = pd.DataFrame({'period': range(397, 445)})
forecasted_demand = arima_model_fit.forecast(steps=len(forecast_dates))
random_walk_predictions['SARIMA'] = np.array(forecasted_demand)

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(data['period'], data['demand'], label='Original Data')
plt.plot(forecast_dates, forecasted_demand, color='green', label='Forecast')
plt.title(model)
plt.xlabel('Period')
plt.ylabel('Demand')
plt.legend()
plt.show()

Print the results (in this case they are quite similar)

In [ ]:
print(random_walk_predictions)

Now using a demand with seasonal and trend patterns

In [ ]:
data = pd.read_csv("trend_season.csv", delimiter=',')
trend_season_predictions = pd.DataFrame()

Applying SARIMA model (seasonal and trend data)

In [ ]:
# Fit auto_arima to automatically select the best ARIMA model
model = auto_arima(data['demand'], seasonal=True, m=12)

# Fit the best ARIMA model
order = model.get_params()['order']
seasonal_order = model.get_params()['seasonal_order']
arima_model = ARIMA(data['demand'], order=order, seasonal_order=seasonal_order)
arima_model_fit = arima_model.fit()

# Generate forecast for the next 12 observations
forecast_dates = pd.DataFrame({'period': range(397, 409)})
forecasted_demand = arima_model_fit.forecast(steps=len(forecast_dates))
trend_season_predictions['SARIMA'] = np.array(forecasted_demand)

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(data['period'], data['demand'], label='Original Data')
plt.plot(forecast_dates, forecasted_demand, color='green', label='Forecast')
plt.title(model)
plt.xlabel('Period')
plt.ylabel('Demand')
plt.legend()
plt.show()


Applying XGBoost model. In this case we need to create lagged demand in the dataframe so, to avoid messing the datrafame 'data' we will upload the data to different name: 'df':

In [ ]:
df = pd.read_csv("trend_season.csv", delimiter=',')

In [ ]:
# Defining the number of lags to use as features
n_lags = 12  
for i in range(1, n_lags + 1):
    df[f'demand_lag_{i}'] = df['demand'].shift(i)

features = ['demand_lag_1', 'demand_lag_2', 'demand_lag_3', 'demand_lag_4', 'demand_lag_5', 'demand_lag_6', 
            'demand_lag_7', 'demand_lag_8', 'demand_lag_9','demand_lag_10', 'demand_lag_11', 'demand_lag_12']

# Lagged demand will be the feature variables
X = df[features]

# Demand will be the target variable
y = df['demand']

# Split dataframes in training and test
X_train = X.iloc[:len(X)-12,:]
X_test = X.iloc[len(X)-12:,:]
y_train = y[:len(y)-12]
y_test = y[len(y)-12:]

# Training an XGBoost model
model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
model.fit(X_train, y_train)

# Make predictions for the next 12 periods using the test set
y_pred = model.predict(X_test)

# Evaluate the model using mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error (MAE):', mae)

# Plot the actual and predicted demand for the test set
plt.plot(range(len(y)), y.values, label='Actual Demand')
plt.plot(range(len(y), len(y)+len(y_pred)), y_pred, label='Predicted Demand')
plt.xlabel('Time')
plt.ylabel('Demand')
plt.legend()
plt.show()


Printing the results

In [ ]:
trend_season_predictions['XGBoost'] = np.array(y_pred)
trend_season_predictions['Actual'] = np.array(y_test)
print(trend_season_predictions)

Calculating the mean absolute error for both models and comparing the results

In [ ]:
mae = mean_absolute_error(trend_season_predictions['SARIMA'], trend_season_predictions['Actual'])
print('Mean Absolute Error (MAE) for SARIMA:', mae)
mae = mean_absolute_error(trend_season_predictions['XGBoost'], trend_season_predictions['Actual'])
print('Mean Absolute Error (MAE) for XGBoost:', mae)